In [14]:
import math

def distance(a, b):
    return math.hypot(*(v2 - v1 for v1, v2 in zip(a, b)))

def dis(x1, y1, x2, y2):
    return math.sqrt((x2-x1)**2 + (y2 - y1)**2)

In [26]:
import matplotlib.image as img
from tqdm.notebook import tqdm
import numpy as np
from PIL import Image as im 
from PIL import ImageFont
from PIL import ImageDraw 

first_dir = 'cells_2'
micro = 'micro'
cnt = '20'
file_name_base = 'prod_'

files = [i for i in range(1, 11)]

for file_n in tqdm(files):
    
    file_name_all = f'./{first_dir}/{micro}/{cnt}' + '/' + file_name_base + str(file_n) + '.jpg'
    image = img.imread(file_name_all)
    image = np.array(image)

    print('================ FILE ', file_n, '===================')
    print('Image shape ==', image.shape)
    
    file_name_2 = file_name_all[:1] + file_name_all[1:].split('.')[0] + '_1.' + file_name_all[1:].split('.')[1]

    image_1 = img.imread(file_name_2)
    image_1 = np.array(image_1)

    print('Image mask shape ==', image_1.shape)
    
    points_cell = []
    for i in range(image_1.shape[0]):
        for j in range(image_1.shape[1]):
            green = np.array([0, 255, 0])
            h_green = distance(np.array(image_1[i][j]), green)
            if h_green < 50:
                points_cell.append([i, j])
                
    
    print('Mask pixel size = ', len(points_cell))
    
    points_d = []
    points_t = []

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            yellow = np.array([255, 255, 0])
            green = np.array([0, 255, 0])
            h_green = distance(np.array(image[i][j]), green)
            h_purple = distance(np.array(image[i][j]), yellow)
        
            if h_green < 50:
                points_t.append([i, j])
            elif h_purple < 50:
                points_d.append([i, j])

                

    print('Points for target = ', len(points_t), ', Points for diametr = ', len(points_d))
    
    nearest_points = []

    for i in range(len(points_d)):
        for j in range(i+1, len(points_d)):
            if dis(points_d[i][0], points_d[i][1], points_d[j][0], points_d[j][1]) <= 10:
                nearest_points.append(i)
            
    points_d = [points_d[i] for i in range(len(points_d)) if i not in nearest_points]
    
    print(f'Diametr {len(points_d)} points = ', points_d)
    
    max_y = 0
    min_y = image.shape[1]
    max_x = 0
    min_x = image.shape[0]

    for i in points_d:
        if i[0] >= max_x:
            max_x = i[0]
            right = i
        if i[0] <= min_x:
            min_x = i[0]
            left = i
        if i[1] >= max_y:
            max_y = i[1]
            down = i
        if i[1] <= min_y:
            min_y = i[1]
            up = i

    diam = int((dis(up[0], up[1], down[0], down[1]) + dis(left[0], left[1], right[0], right[1])) / 2)
    print('Total D = ', diam)
    
    um_for_pix = (15.4 * 1000) / diam
    print(f'µm for pixel == {um_for_pix}')

    diam_sf = []
    centers = []

    x, y = np.mean(np.array(points_t), axis=0)
    x = int(x)
    y = int(y)
    
    print('Подсчет диаметра таргера')

    for a in points_t:
        for b in points_t:
            for p in [[x, y],
                      [x+1, y+1], [x+2, y+2],
                      [x-1, y-1], [x-2, y-2],
                      [x+1, y-1], [x-1, y+1],
                      [x-1, y], [x, y-1],
                      [x+1, y], [x, y+1]]:
            
                first_case = ((p[0] - a[0]) * (b[1] - a[1]) - (b[0] - a[0]) * (p[1] - a[1]) == 0)
                distance_case = (distance(a, b) > 10)
                points_case = (p[0] >= a[0] and p[0] <= b[0] and p[1] <= a[1] and p[1] >= b[1])
                
                if first_case and distance_case and points_case:
                    
                    d = dis(a[0], a[1], b[0], b[1])
                
                    if d > 0:
                        diam_sf.append(d)
                        centers.append([(a[0] + b[0]) / 2, (a[1] + b[1]) / 2])
                    if d < 5:
                        print('!!!! - ', a, b, p)
                        
    print(f'Диаметр таргета высчитан по {len(diam_sf)} точкам')
        
    r = int(np.mean(diam_sf) / 2)
    x, y = np.mean(np.array(points_t), axis=0)
    x = int(x)
    y = int(y)

    x_min = min([i[0] for i in points_t])
    y_min = min([i[1] for i in points_t])
    x_max = max([i[0] for i in points_t])
    y_max = max([i[1] for i in points_t])

    points_area = 0
    points_in_c = 0
    
    print('Подсчет округлости')

    for i in range(x_min, x_max + 1):
        for j in range(y_min, y_max + 1):
            if points_cell.count([i, j]) > 0:
                points_area += 1
                if (r ** 2) >= ((x - i) ** 2 + (y - j) ** 2):
                    points_in_c += 1
                
    print(f'Диаметр посчитан на {len(diam_sf)} отрезках')
    print('Диаметр = ', round(np.mean(diam_sf) * um_for_pix / 1000, 3), 'mm')
    print('Диаметр = ', round(np.mean(diam_sf) * um_for_pix) , 'µm')
    print('Круглость = ', round(points_in_c / points_area * 100, 1), '%')

    result_txt = f'''
Диаметр = {round(np.mean(diam_sf) * um_for_pix / 1000, 3)} mm
Диаметр = {round(np.mean(diam_sf) * um_for_pix)} µm
Круглость = {round(points_in_c / points_area * 100, 1)} %
Отрезков диаметра таргета = {len(diam_sf)} штук
Диаметр окружности в пикселях =  {diam}
Точки диаметра: {points_d}
µm for pixel == {um_for_pix}
'''
    
    # Проверка

    image_copy = image.copy()

    x, y = np.mean(np.array(points_t), axis=0)
    x = int(x)
    y = int(y)

    for i in range(image_copy.shape[0]):
        for j in range(image_copy.shape[1]):
            if i >= x-3 and i <= x+3 and j >= y-3 and j <= y+3:
                image_copy[i][j] = [0, 0, 0]

    for i, j in points_t:
        image_copy[i][j] = [255, 0, 0]
    
    for i, j in points_d:
        image_copy[i][j] = [255, 0, 0]
        image_copy[i+1][j] = [255, 0, 0]
        image_copy[i][j+1] = [255, 0, 0]
        image_copy[i-1][j] = [255, 0, 0]
        image_copy[i][j-1] = [255, 0, 0]
    
    
    r = int(np.mean(diam_sf) / 2)
    x, y = np.mean(np.array(points_t), axis=0)
    x = int(x)
    y = int(y)

    for i in range(x - r, x + r + 1):
        for j in range(y - r, y + r + 1):
            r_2 = (r ** 2)
            circle = ((x - i) ** 2 + (y - j) ** 2)
            if abs(r_2 - circle) <= 3:
                image_copy[i][j] = [255, 255, 255]

    k = im.fromarray(image_copy) 
    # Проверка
    
    print('Проверка прошла')
    
    
    x = max([i[0] for i in points_t]) + 25
    y = min([i[1] for i in points_t]) + 25
    color = [0, 36, 255]

    for i in range(y, int(y + 1000 / um_for_pix)):
        image[x][i] = color
        image[x+1][i] = color
        image[x+2][i] = color
    
    image[x-1][y] = color
    image[x-1][y] = color
    image[x-2][y] = color
    image[x-2][y] = color
    image[x+3][y] = color
    image[x+3][y] = color
    image[x+4][y] = color
    image[x+4][y] = color

    image[x-1][int(y + 1000 / um_for_pix)] = color
    image[x-1][int(y + 1000 / um_for_pix)] = color
    image[x-2][int(y + 1000 / um_for_pix)] = color
    image[x-2][int(y + 1000 / um_for_pix)] = color
    image[x+3][int(y + 1000 / um_for_pix)] = color
    image[x+3][int(y + 1000 / um_for_pix)] = color
    image[x+4][int(y + 1000 / um_for_pix)] = color
    image[x+4][int(y + 1000 / um_for_pix)] = color
    
    f = im.fromarray(image) 
    
    draw = ImageDraw.Draw(f)
    font = ImageFont.truetype("./arial.ttf", 22);
    draw.text((y, x + 10),"1000 µm = 1 mm",(0, 36, 255),font=font)
    
    print('Закончили, сохраняем')
    
    f.save(f'./{first_dir}/{micro}/{cnt}' + '/results/' + file_name_base + str(file_n) + '.jpg')
    k.save(f'./{first_dir}/{micro}/{cnt}' + '/results/' + file_name_base + str(file_n) + '_kontrol.jpg')
    file = open(f'./{first_dir}/{micro}/{cnt}' + '/results/' + file_name_base + str(file_n) + '.txt', "w")
    file.write(result_txt)
    file.close()
           
    print('Все ок, сохранил')
    print('\n\n\n')

================ FILE  1 ===================
Image shape == (1217, 1256, 3)
Image mask shape == (1217, 1256, 3)
Mask pixel size =  81830
Points for target =  1367 , Points for diametr =  11
Diametr 4 points =  [[65, 546], [560, 1187], [621, 39], [1164, 610]]
Total D =  1125
µm for pixel == 13.688888888888888
Подсчет диаметра таргера
Диаметр таргета высчитан по 133 точкам
Подсчет округлости
Диаметр посчитан на 133 отрезках
Диаметр =  4.543 mm
Диаметр =  4543 µm
Круглость =  85.5 %
Проверка прошла
Закончили, сохраняем
Все ок, сохранил




================ FILE  2 ===================
Image shape == (1267, 1256, 3)
Image mask shape == (1267, 1256, 3)
Mask pixel size =  82142
Points for target =  1448 , Points for diametr =  9
Diametr 4 points =  [[53, 557], [578, 1197], [628, 54], [1200, 598]]
Total D =  1145
µm for pixel == 13.449781659388647
Подсчет диаметра таргера
Диаметр таргета высчитан по 107 точкам
Подсчет округлости
Диаметр посчитан на 107 отрезках
Диаметр =  3.907 mm
Диаметр =  3

In [1]:
from notebook.services.config import ConfigManager
c = ConfigManager()
c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseBrackets": True}}})

ModuleNotFoundError: No module named 'notebook.services'